# Natural Language Processing

J.F. Omhover, with thanks to Mari Pierce-Quinonez for some great enhancements.

Updated for Python 3.6 by Miles Erickson

### Requirements

You need to install the `nltk` module:

```
conda install nltk
```

This module will need corpora that you need to download. This can take a very long time, for simplicity here's the minimal corpora for this lecture. Run these from `ipython` or a jupyter notebook:

In [27]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_treebank_pos_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jackbennetto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jackbennetto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jackbennetto/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /Users/jackbennetto/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!


True

### General Introduction

Natural Language Processing is a subfield of machine learning focused on making sense of text. Text is inherently unstructured and has all sorts of tricks required for converting (vectorizing) text into a format that a machine learning algorithm can interpret. It is called Processing for a reason - most of what we'll be covering during this morning session are Data Processing operations that make it possible to plug test into other ML algorithms.

### Overview of nlp

Natural language processing is concerned with understanding text using computation. People working within the field are often concerned with:
- Information retrieval. How do you find a document or a particular fact within a document?
- Document classification. What is the document about amongst mutually exclusive categories?
- Machine translation. How do you write an English phrase in Chinese? Think of Google translate.
- Sentiment analysis. Was a product review positive or negative?
Natural language processing is a huge field and we will just touch on some of the concepts.

### Objectives

- Name and describe the steps necessary for processing text in machine learning.
- Implement a Natural Language Processing pipeline.
- Explain the cosine similarity measure and why it is used in NLP.

# Text Featurization part 1 : Bags of Words

This Walkthrough will lead us from raw documents to bag-of-words representations using **Natural Language Processing** functions.

In our case, this walkthrough is a preliminary step of a pipeline for **indexing** documents.

The ultimate goal of **indexing** is to create a **signature** (vector) for each document.

This **signature** will be used for relating documents one to the other (and find out similar clusters of documents), or for mining underlying relations between concepts.

<img src="img/pipeline-walkthrough1.png" width="70%"/>

## 0. Text sources and possible text mining inputs

In [28]:
paragraph = "My mother drove me to the airport with the windows rolled down. It was seventy-five degrees in Phoenix, the sky a perfect, cloudless blue. I was wearing my favorite shirt – sleeveless, white eyelet lace; I was wearing it as a farewell gesture. My carry-on item was a parka. In the Olympic Peninsula of northwest Washington State, a small town named Forks exists under a near-constant cover of clouds. It rains on this inconsequential town more than any other place in the United States of America. It was from this town and its gloomy, omnipresent shade that my mother escaped with me when I was only a few months old. It was in this town that I’d been compelled to spend a month every summer until I was fourteen. That was the year I finally put my foot down; these past three summers, my dad, Charlie, vacationed with me in California for two weeks instead."

paragraph

'My mother drove me to the airport with the windows rolled down. It was seventy-five degrees in Phoenix, the sky a perfect, cloudless blue. I was wearing my favorite shirt – sleeveless, white eyelet lace; I was wearing it as a farewell gesture. My carry-on item was a parka. In the Olympic Peninsula of northwest Washington State, a small town named Forks exists under a near-constant cover of clouds. It rains on this inconsequential town more than any other place in the United States of America. It was from this town and its gloomy, omnipresent shade that my mother escaped with me when I was only a few months old. It was in this town that I’d been compelled to spend a month every summer until I was fourteen. That was the year I finally put my foot down; these past three summers, my dad, Charlie, vacationed with me in California for two weeks instead.'

In [29]:
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode()

input_string = remove_accents(paragraph)

# 1. Creating bag-of-words for each document

## 1.1. Tokenize document

**"Tokenize"** means creating "tokens" which are atomic units of the text. These tokens are usually words we extract from the document by splitting it (using punctuations as a separator). We can also consider sentences as tokens (and words as sub-tokens of sentences).

### nltk.tokenize.sent_tokenize

In [30]:
from nltk.tokenize import sent_tokenize

sent_tokens = sent_tokenize(input_string)

sent_tokens

['My mother drove me to the airport with the windows rolled down.',
 'It was seventy-five degrees in Phoenix, the sky a perfect, cloudless blue.',
 'I was wearing my favorite shirt  sleeveless, white eyelet lace; I was wearing it as a farewell gesture.',
 'My carry-on item was a parka.',
 'In the Olympic Peninsula of northwest Washington State, a small town named Forks exists under a near-constant cover of clouds.',
 'It rains on this inconsequential town more than any other place in the United States of America.',
 'It was from this town and its gloomy, omnipresent shade that my mother escaped with me when I was only a few months old.',
 'It was in this town that Id been compelled to spend a month every summer until I was fourteen.',
 'That was the year I finally put my foot down; these past three summers, my dad, Charlie, vacationed with me in California for two weeks instead.']

### nltk.tokenize.word_tokenize

In [33]:
from nltk.tokenize import word_tokenize

tokens = [word_tokenize(sent) for sent in sent_tokens]

list(enumerate(tokens))

[(0,
  ['My',
   'mother',
   'drove',
   'me',
   'to',
   'the',
   'airport',
   'with',
   'the',
   'windows',
   'rolled',
   'down',
   '.']),
 (1,
  ['It',
   'was',
   'seventy-five',
   'degrees',
   'in',
   'Phoenix',
   ',',
   'the',
   'sky',
   'a',
   'perfect',
   ',',
   'cloudless',
   'blue',
   '.']),
 (2,
  ['I',
   'was',
   'wearing',
   'my',
   'favorite',
   'shirt',
   'sleeveless',
   ',',
   'white',
   'eyelet',
   'lace',
   ';',
   'I',
   'was',
   'wearing',
   'it',
   'as',
   'a',
   'farewell',
   'gesture',
   '.']),
 (3, ['My', 'carry-on', 'item', 'was', 'a', 'parka', '.']),
 (4,
  ['In',
   'the',
   'Olympic',
   'Peninsula',
   'of',
   'northwest',
   'Washington',
   'State',
   ',',
   'a',
   'small',
   'town',
   'named',
   'Forks',
   'exists',
   'under',
   'a',
   'near-constant',
   'cover',
   'of',
   'clouds',
   '.']),
 (5,
  ['It',
   'rains',
   'on',
   'this',
   'inconsequential',
   'town',
   'more',
   'than',
   'any

### lower

In [34]:
import string

tokens_lower = [[word.lower() for word in sent]
                 for sent in tokens]

## 1.2. Filtering stopwords (and punctuation)

**Stopwords** are words that should be stopped at this step because they do not carry much information about the actual meaning of the document. Usually, they are "common" words you use. You can find lists of such **stopwords** online, or embedded within the NLTK library.

### Using your own stop list

In [41]:
from nltk.corpus import stopwords

stopwords_ = set(stopwords.words('english'))

print("--- stopwords in english: {}".format(stopwords_))

--- stopwords in english: {'too', 'just', 'doing', 'her', "couldn't", "haven't", 'those', 'up', 'than', 'who', 'ain', 'off', 'to', 'most', "weren't", 'did', 'o', 'on', 'of', 'each', "don't", "hadn't", "you'll", 'has', 'some', 'y', "hasn't", "you've", 'them', 'or', 'same', 'while', 'for', 'against', "isn't", 'any', 'himself', 'yourselves', 'been', 'we', 'it', 'weren', "shan't", 'don', "she's", 'such', 'an', 'myself', 'once', "shouldn't", 'won', 'couldn', 'about', 'didn', 'then', 'a', 'shouldn', 'no', 'hasn', 'can', 'he', 'further', 'but', 'had', 'needn', "won't", 'that', 'down', 'i', 'our', 'if', 'isn', 'into', 'until', 'above', "wouldn't", 'again', "mustn't", 'both', 'what', "that'll", 'doesn', 'your', 'whom', 'in', 'hers', 'me', 'at', 'during', 'wouldn', 'when', 'him', 'and', 'below', 'ourselves', 's', 'which', 'mightn', "it's", 'with', 'being', 're', 'wasn', 'more', 'was', 'because', 'as', 'should', 'itself', "you're", 'having', 'you', 'from', 'how', 'will', 'aren', 'not', 'their', '

In [42]:
# list found at http://www.textfixer.com/resources/common-english-words.txt
# 'not' has been removed (do you know why ?)

stopwords_ = set("a,able,about,across,after,all,almost,also,am,among,an,and,any,\
are,as,at,be,because,been,but,by,can,could,dear,did,do,does,either,\
else,ever,every,for,from,get,got,had,has,have,he,her,hers,him,his,\
how,however,i,if,in,into,is,it,its,just,least,let,like,likely,may,\
me,might,most,must,my,neither,no,of,off,often,on,only,or,other,our,\
own,rather,said,say,says,she,should,since,so,some,than,that,the,their,\
them,then,there,these,they,this,tis,to,too,twas,us,wants,was,we,were,\
what,when,where,which,while,who,whom,why,will,with,would,yet,you,your]".split(','))

print("--- stopwords in english: {}".format(stopwords_))

--- stopwords in english: {'too', 'just', 'neither', 'her', 'than', 'who', 'off', 'to', 'most', 'did', 'on', 'of', 'has', 'some', 'or', 'them', 'get', 'while', 'for', 'every', 'any', 'been', 'we', 'must', 'it', 'ever', 'tis', 'either', 'an', 'else', 'about', 'then', 'a', 'no', 'can', 'he', 'but', 'had', 'least', 'since', 'that', 'i', 'our', 'if', 'however', 'into', 'could', 'yet', 'might', 'what', 'in', 'whom', 'like', 'hers', 'me', 'at', 'dear', 'said', 'when', 'him', 'and', 'wants', 'among', 'almost', 'which', 'say', 'with', 'because', 'should', 'as', 'was', 'may', 'twas', 'from', 'likely', 'how', 'you', 'will', 'their', 'does', 'us', 'they', 'often', 'my', 'is', 'other', 'have', 'why', 'am', 'also', 'are', 'across', 'rather', 'all', 'be', 'his', 'your]', 'let', 'so', 'after', 'where', 'able', 'would', 'there', 'were', 'own', 'the', 'only', 'these', 'she', 'this', 'got', 'do', 'by', 'says', 'its'}


We also need to filter punctuation tokens: tokens made of punctuation marks. We can find a list of those punctuations in `string.punctuation`.

In [44]:
import string

punctuation_ = set(string.punctuation)
print("--- punctuation: {}".format(string.punctuation))

def filter_tokens(sent):
    return([w for w in sent if not w in stopwords_ and not w in punctuation_])

tokens_filtered = list(map(filter_tokens, tokens_lower))

for sent in tokens_filtered:
    print("--- sentence tokens: {}".format(sent))

--- punctuation: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
--- sentence tokens: ['mother', 'drove', 'airport', 'windows', 'rolled', 'down']
--- sentence tokens: ['seventy-five', 'degrees', 'phoenix', 'sky', 'perfect', 'cloudless', 'blue']
--- sentence tokens: ['wearing', 'favorite', 'shirt', 'sleeveless', 'white', 'eyelet', 'lace', 'wearing', 'farewell', 'gesture']
--- sentence tokens: ['carry-on', 'item', 'parka']
--- sentence tokens: ['olympic', 'peninsula', 'northwest', 'washington', 'state', 'small', 'town', 'named', 'forks', 'exists', 'under', 'near-constant', 'cover', 'clouds']
--- sentence tokens: ['rains', 'inconsequential', 'town', 'more', 'place', 'united', 'states', 'america']
--- sentence tokens: ['town', 'gloomy', 'omnipresent', 'shade', 'mother', 'escaped', 'few', 'months', 'old']
--- sentence tokens: ['town', 'id', 'compelled', 'spend', 'month', 'summer', 'until', 'fourteen']
--- sentence tokens: ['year', 'finally', 'put', 'foot', 'down', 'past', 'three', 'summers', 'dad', 'charl

## 1.3. Stemming and lemmatization

**Stemming** means reducing each word to a **stem**. That is, reducing each word in all its diversity to a root common to all its variants. **Lemmatizing** is similar, but is slower and has a greater understanding of the words in the language.

In [50]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

sentence_number = 0

stemmer_porter = PorterStemmer()
tokens_stemporter = [list(map(stemmer_porter.stem, sent)) for sent in tokens_filtered]
print("--- sentence tokens (porter): {}".format(tokens_stemporter[sentence_number]))

stemmer_snowball = SnowballStemmer('english')
tokens_stemsnowball = [list(map(stemmer_snowball.stem, sent)) for sent in tokens_filtered]
print("--- sentence tokens (snowball): {}".format(tokens_stemsnowball[sentence_number]))

lemmatizer = WordNetLemmatizer()
tokens_lemmatize = [list(map(lemmatizer.lemmatize, sent)) for sent in tokens_filtered]
print("--- sentence tokens (lemmatize): {}".format(tokens_lemmatize[sentence_number]))

--- sentence tokens (porter): ['mother', 'drove', 'airport', 'window', 'roll', 'down']
--- sentence tokens (snowball): ['mother', 'drove', 'airport', 'window', 'roll', 'down']
--- sentence tokens (lemmatize): ['mother', 'drove', 'airport', 'window', 'rolled', 'down']


## 1.4. N-Grams

An n-gram is a sequence of (generally consecutive) words that are treated as a single token.

In [53]:
from nltk.util import ngrams

list(ngrams(tokens_stemsnowball[0],2))

[('mother', 'drove'),
 ('drove', 'airport'),
 ('airport', 'window'),
 ('window', 'roll'),
 ('roll', 'down')]

In [54]:
from nltk.util import ngrams

def join_sent_ngrams(input_tokens, n):
    # first add the 1-gram tokens
    ret_list = list(input_tokens)
    
    #then for each n
    for i in range(2,n+1):
        # add each n-grams to the list
        ret_list.extend(['-'.join(tgram) for tgram in ngrams(input_tokens, i)])
    
    return(ret_list)

tokens_ngrams = list(map(lambda x : join_sent_ngrams(x, 3), tokens_stemporter))

print("--- sentence tokens: {}".format(tokens_ngrams[0]))

--- sentence tokens: ['mother', 'drove', 'airport', 'window', 'roll', 'down', 'mother-drove', 'drove-airport', 'airport-window', 'window-roll', 'roll-down', 'mother-drove-airport', 'drove-airport-window', 'airport-window-roll', 'window-roll-down']


## 1.5. Part-of-Speech tagging

This is an alternative process that relies on machine learning to tag each word in a sentence with its function. In libraries such as NLTK, there are embedded tools to do that. Tags detected depend on the corpus used for training. In NLTK, the function `nltk.pos_tag()` uses the [Penn Treebank](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html).

### nltk.pos_tag

In [55]:
from nltk import pos_tag

sent_tags = list(map(pos_tag, tokens))

for sent in sent_tags:
    print("--- sentence tags: {}".format(sent))

--- sentence tags: [('My', 'PRP$'), ('mother', 'NN'), ('drove', 'VBD'), ('me', 'PRP'), ('to', 'TO'), ('the', 'DT'), ('airport', 'NN'), ('with', 'IN'), ('the', 'DT'), ('windows', 'NNS'), ('rolled', 'VBD'), ('down', 'RB'), ('.', '.')]
--- sentence tags: [('It', 'PRP'), ('was', 'VBD'), ('seventy-five', 'JJ'), ('degrees', 'NNS'), ('in', 'IN'), ('Phoenix', 'NNP'), (',', ','), ('the', 'DT'), ('sky', 'NN'), ('a', 'DT'), ('perfect', 'JJ'), (',', ','), ('cloudless', 'JJ'), ('blue', 'NN'), ('.', '.')]
--- sentence tags: [('I', 'PRP'), ('was', 'VBD'), ('wearing', 'VBG'), ('my', 'PRP$'), ('favorite', 'JJ'), ('shirt', 'NN'), ('sleeveless', 'NN'), (',', ','), ('white', 'JJ'), ('eyelet', 'NN'), ('lace', 'NN'), (';', ':'), ('I', 'PRP'), ('was', 'VBD'), ('wearing', 'VBG'), ('it', 'PRP'), ('as', 'IN'), ('a', 'DT'), ('farewell', 'NN'), ('gesture', 'NN'), ('.', '.')]
--- sentence tags: [('My', 'PRP$'), ('carry-on', 'JJ'), ('item', 'NN'), ('was', 'VBD'), ('a', 'DT'), ('parka', 'NN'), ('.', '.')]
--- senten

Note that it isn't perfect.

In [62]:
from nltk import pos_tag

print(pos_tag('Time flies like an arrow'.split()))
print(pos_tag('Fruit flies like a banana'.split()))

[('Time', 'NNP'), ('flies', 'NNS'), ('like', 'IN'), ('an', 'DT'), ('arrow', 'NN')]
[('Fruit', 'NNP'), ('flies', 'VBZ'), ('like', 'IN'), ('a', 'DT'), ('banana', 'NN')]


Let's filter verbs !

In [63]:
for sent in sent_tags:
    tags_filtered = [t for t in sent if t[1].startswith('VB')]
    print("--- verbs:\n{}".format(tags_filtered))

--- verbs:
[('drove', 'VBD'), ('rolled', 'VBD')]
--- verbs:
[('was', 'VBD')]
--- verbs:
[('was', 'VBD'), ('wearing', 'VBG'), ('was', 'VBD'), ('wearing', 'VBG')]
--- verbs:
[('was', 'VBD')]
--- verbs:
[('named', 'VBN'), ('exists', 'VBZ')]
--- verbs:
[('rains', 'VBZ')]
--- verbs:
[('was', 'VBD'), ('escaped', 'VBD'), ('was', 'VBD')]
--- verbs:
[('was', 'VBD'), ('been', 'VBN'), ('compelled', 'VBN'), ('spend', 'VB'), ('was', 'VBD')]
--- verbs:
[('was', 'VBD'), ('put', 'VBD'), ('vacationed', 'VBD')]


In [64]:
from nltk import RegexpParser

grammar = r"""
  NPB: {<DT|PP\$>?<JJ|NN|,>*<NN>}   # chunk determiner/possessive, adjectives and noun
      {<NNP>+}                # chunk sequences of proper nouns
  V2V: {<V.*> <TO> <V.*>}
"""

cp = RegexpParser(grammar)
result = cp.parse(sent_tags[1])

#print result

for sent in sent_tags:
    tree = cp.parse(sent)
    for subtree in tree.subtrees():
        if subtree.label() == 'NPB': print(subtree)
        if subtree.label() == 'V2V': print(subtree)

(NPB mother/NN)
(NPB the/DT airport/NN)
(NPB Phoenix/NNP)
(NPB the/DT sky/NN)
(NPB a/DT perfect/JJ ,/, cloudless/JJ blue/NN)
(NPB
  favorite/JJ
  shirt/NN
  sleeveless/NN
  ,/,
  white/JJ
  eyelet/NN
  lace/NN)
(NPB a/DT farewell/NN gesture/NN)
(NPB carry-on/JJ item/NN)
(NPB a/DT parka/NN)
(NPB Olympic/NNP Peninsula/NNP)
(NPB Washington/NNP State/NNP)
(NPB a/DT small/JJ town/NN)
(NPB Forks/NNP)
(NPB a/DT near-constant/JJ cover/NN)
(NPB this/DT inconsequential/JJ town/NN)
(NPB any/DT other/JJ place/NN)
(NPB United/NNP)
(NPB America/NNP)
(NPB this/DT town/NN)
(NPB gloomy/NN ,/, omnipresent/NN shade/NN)
(NPB mother/NN)
(NPB this/DT town/NN)
(NPB Id/NNP)
(V2V compelled/VBN to/TO spend/VB)
(NPB a/DT month/NN)
(NPB every/DT summer/NN)
(NPB the/DT year/NN)
(NPB foot/NN)
(NPB dad/NN)
(NPB Charlie/NNP)
(NPB California/NNP)


# Text Featurization part 2 : Indexing Bag-of-Words into a vector table

This Walkthrough will lead us from bag-of-words representations of documents to **vector signatures** (indexes) using the **TF-IDF** formula.

The ultimate goal of **indexing** is to create a **vector representation** (signature) for each document. This vector representation will be used for:

- mine the features that can caracterize classes of documents (supervised learning using **labels**)
- mine the documents that have similar features to establish trends (unsupervised learning).

To do that, we need:
- a fixed number of features
- a quantitative value for each feature.

The number of features is given by the vocabulary over the corpus: the set of all possible words (tokens) found in all documents.

The quantitative value is given, for each doc, by counting the occurences of each of these words in the doc and by using a TF-IDF formula.

<img src="img/pipeline-walkthrough2.png" width="70%"/>

## 0. Loading some input data from the Amazon Reviews

To try this indexing walkthrough, we will get 5 reviews from the Amazon Reviews dataset. We will apply a function for extracting bag-of-words representations from these 5 documents.

In [66]:
import os               # for environ variables in Part 3
from nlp_pipeline import extract_bow_from_raw_text
import json

docs = []
with open('./reviews.json', 'r') as data_file:    
    for line in data_file:
        docs.append(json.loads(line))

# extracting bows
bows = list(map(lambda row: extract_bow_from_raw_text(row['reviewText']), docs))

# displaying bows
for i in range(len(docs)):
    print("\n--- review: {}".format(docs[i]['reviewText']))
    print("--- bow: {}".format(bows[i]))


--- review: Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing,
--- bow: ['b', 'filter', 'pop', 'record', 'more', 'crisp', 'lowest', 'price', 'filter', 'amazon', 'same', 'price']

--- review: The product does exactly as it should and is quite affordable.I did not realized it was double screened until it arrived, so it was even better than I had expected.As an added bonus, one of the screens carries a small hint of the smell of an old grape candy I used to buy, so for reminiscent's sake, I cannot stop putting the pop filter next to my nose and smelling it after recording. :DIf you needed a pop filter, this will work just as well as the expensive ones, and it may even come with a pleasing aroma like mine did!Buy this product! :]
--- bow: ['b', 'product', 'doubl', 'better', 

# 1. Indexing Bag of Words into a Vector Matrix using Term Frequency / Inverse Document Frequency
The ultimate goal of indexing is to create a vector representation (signature) for each document. This vector representation will be used for:
mine the features that can caracterize classes of documents (supervised learning using labels)
mine the documents that have similar features to establish trends (unsupervised learning).
To do that, we need:
- a fixed number of features
- a quantitative value for each feature.

The number of features is given by the vocabulary over the corpus: the set of all possible words (tokens) found in all documents.

The quantitative value is given, for each doc, by counting the occurences of each of these words in the doc and by using a TF-IDF formula.

## 1.1 Term Frequency

The number of times a term occurs in a specific document:

$tf(term,document) = \frac{\# \ of \ times \ a \ term \ appears \ in \ a \ document}{\#\ of\ terms\ in\ the\ document|}$

In [67]:
from collections import Counter

# term occurence = counting distinct words in each bag
term_occ = list(map(lambda bow : Counter(bow), bows))

# term frequency = occurences over length of bag
term_freq = list()
for i in range(len(docs)):
    term_freq.append( {k: (v / float(len(bows[i])))
                       for k, v in term_occ[i].items()} )

# displaying occurences
for i in range(len(docs)):
    print("\n--- review: {}".format(docs[i]['reviewText']))
    print("--- bow: {}".format(bows[i]))
    print("--- term_occ: {}".format(term_occ[i]))
    print("--- term_freq: {}".format(term_freq[i]))


--- review: Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing,
--- bow: ['b', 'filter', 'pop', 'record', 'more', 'crisp', 'lowest', 'price', 'filter', 'amazon', 'same', 'price']
--- term_occ: Counter({'filter': 2, 'price': 2, 'b': 1, 'pop': 1, 'record': 1, 'more': 1, 'crisp': 1, 'lowest': 1, 'amazon': 1, 'same': 1})
--- term_freq: {'b': 0.08333333333333333, 'filter': 0.16666666666666666, 'pop': 0.08333333333333333, 'record': 0.08333333333333333, 'more': 0.08333333333333333, 'crisp': 0.08333333333333333, 'lowest': 0.08333333333333333, 'price': 0.16666666666666666, 'amazon': 0.08333333333333333, 'same': 0.08333333333333333}

--- review: The product does exactly as it should and is quite affordable.I did not realized it was double screened until it arrived, so it was even b

## 1.2. Obtaining document frequencies

$df(term,corpus) = \frac{ \# \ of \ documents \ that \ contain \ a \ term}{ \# \ of \ documents \ in \ the \ corpus}$


In [68]:
# document occurence = number of documents having this word
# term frequency = occurences over length of bag

doc_occ = Counter( [word for bow in bows for word in set(bow)] )

# document frequency = occurences over length of corpus
doc_freq = {k: (v / float(len(docs)))
            for k, v in doc_occ.items()}

# displaying vocabulary
print("\n--- full vocabulary: {}".format(doc_occ))
print("\n--- doc freq: {}".format(doc_freq))


--- full vocabulary: Counter({'pop': 5, 'filter': 4, 'b': 3, 'doubl': 2, 'screen': 2, 'clamp': 2, 'record': 1, 'price': 1, 'same': 1, 'crisp': 1, 'lowest': 1, 'amazon': 1, 'more': 1, 'reminisc': 1, 'expens': 1, 'better': 1, 'sake': 1, 'nose': 1, 'dif': 1, 'old': 1, ']': 1, 'hint': 1, 'ad': 1, 'mine': 1, 'candi': 1, 'grape': 1, 'bonus': 1, 'smell': 1, 'small': 1, 'product': 1, 'one': 1, 'aroma': 1, 'high': 1, 'enough': 1, 'reduct': 1, 'frequenc': 1, 'littl': 1, 'mount': 1, 'goos': 1, 'breath': 1, 'cloth': 1, 'secur': 1, 'neck': 1, 'devic': 1, 'voic': 1, 'primari': 1, 'coax': 1, 'job': 1, 'color': 1, 'attach': 1, 'notic': 1, "b'the": 1, 'metal': 1, 'volum': 1, 'mike': 1, 'sound': 1, 'abl': 1, 'protect': 1, 'prevent': 1, 'thing': 1, 'gooseneck': 1, 'care': 1, 'posit': 1, 'windscreen': 1, 'mxl': 1, 'mic': 1, "b'nice": 1, 'great': 1, 'studio': 1, 'perform': 1, 'vocal': 1})

--- doc freq: {'record': 0.2, 'pop': 1.0, 'price': 0.2, 'same': 0.2, 'crisp': 0.2, 'lowest': 0.2, 'b': 0.6, 'filter':

## 1.3 Creating the vocabulary for indexing

In [69]:
# the minimum document frequency (in proportion of the length of the corpus)
min_df = 0.3

# filtering items to obtain the vocabulary
vocabulary = [ k for k,v in doc_freq.items() if v >= min_df ]

# print vocabulary
print ("-- vocabulary (len={}): {}".format(len(vocabulary),vocabulary))

-- vocabulary (len=6): ['pop', 'b', 'filter', 'doubl', 'screen', 'clamp']


## 1.4 the TFIDF vector

Words might show up a lot in individual documents, but their relevace is less important if they're in every document! We need to take into account words that show up everywhere and reduce their relative importance. The document frequency does exactly that:

$df(term,corpus) = \frac{ \# \ of \ documents \ that \ contain \ a \ term}{ \# \ of \ documents \ in \ the \ corpus}$

The inverse document frequency is defined in terms of the document frequency as

$idf(term,corpus) = \log{\frac{1}{df(term,corpus)}}$.


TF-IDF is an acronym for the product of two parts: the term frequency tf and what is called the inverse document frequency idf. The term frequency is just the counts in a term frequency vector. 

tf-idf $ = tf(term,document) * idf(term,corpus)$

In [70]:
import numpy as np

# create a dense matrix of vectors for each document
# each vector has the length of the vocabulary
vectors = np.zeros((len(docs),len(vocabulary)))

# fill these vectors with tf-idf values
for i in range(len(docs)):
    for j in range(len(vocabulary)):
        term     = vocabulary[j]
        term_tf  = term_freq[i].get(term, 0.0)   # 0.0 if term not found in doc
        term_idf = np.log(1 + 1 / doc_freq[term]) # smooth formula
        vectors[i,j] = term_tf * term_idf

# displaying results
for i in range(len(docs)):
    print("\n--- review: {}".format(docs[i]['reviewText']))
    print("--- bow: {}".format(bows[i]))
    print("--- tfidf vector: {}".format( vectors[i] ) )
    print("--- tfidf sorted: {}".format( 
            sorted( zip(vocabulary,vectors[i]), key=lambda x:-x[1] )
         ))


--- review: Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing,
--- bow: ['b', 'filter', 'pop', 'record', 'more', 'crisp', 'lowest', 'price', 'filter', 'amazon', 'same', 'price']
--- tfidf vector: [ 0.05776227  0.08173577  0.13515504  0.          0.          0.        ]
--- tfidf sorted: [('filter', 0.13515503603605478), ('b', 0.081735771084310516), ('pop', 0.057762265046662105), ('doubl', 0.0), ('screen', 0.0), ('clamp', 0.0)]

--- review: The product does exactly as it should and is quite affordable.I did not realized it was double screened until it arrived, so it was even better than I had expected.As an added bonus, one of the screens carries a small hint of the smell of an old grape candy I used to buy, so for reminiscent's sake, I cannot stop putting the pop filter 

## 1.5 Sklearn pipeline

In [71]:
corpus = [row['reviewText'] for row in docs]

In [72]:
from sklearn.feature_extraction.text import CountVectorizer

tf = CountVectorizer()

document_tf_matrix = tf.fit_transform(corpus).todense()

print(sorted(tf.vocabulary_))
print(document_tf_matrix)

['able', 'about', 'added', 'affordable', 'after', 'allowing', 'amazon', 'an', 'and', 'are', 'aroma', 'arrived', 'as', 'attached', 'attaches', 'avoid', 'better', 'block', 'blocks', 'bonus', 'breath', 'but', 'buy', 'candy', 'cannot', 'careful', 'carries', 'clamp', 'cloth', 'coaxing', 'coloration', 'come', 'crisp', 'despite', 'device', 'did', 'dif', 'does', 'double', 'eliminate', 'enough', 'even', 'exactly', 'expected', 'expensive', 'filter', 'filters', 'for', 'frequencies', 'gets', 'goose', 'gooseneck', 'grape', 'great', 'had', 'here', 'high', 'hint', 'hold', 'honestly', 'if', 'in', 'is', 'it', 'job', 'just', 'keep', 'lets', 'like', 'little', 'looks', 'lowest', 'marginally', 'may', 'metal', 'mic', 'might', 'mike', 'mine', 'more', 'mount', 'much', 'mxl', 'my', 'neck', 'needed', 'needs', 'next', 'nice', 'no', 'nose', 'not', 'noticeable', 'now', 'of', 'old', 'on', 'one', 'ones', 'only', 'or', 'otherwise', 'out', 'pass', 'performs', 'pleasing', 'pop', 'popping', 'pops', 'position', 'position

In [73]:
from math import log

def idf(frequency_matrix):
    df =  float(len(document_tf_matrix)) / sum(frequency_matrix > 0)
    return [log(i) for i in df.getA()[0]]
print(sorted(tf.vocabulary_))
print(idf(document_tf_matrix))

['able', 'about', 'added', 'affordable', 'after', 'allowing', 'amazon', 'an', 'and', 'are', 'aroma', 'arrived', 'as', 'attached', 'attaches', 'avoid', 'better', 'block', 'blocks', 'bonus', 'breath', 'but', 'buy', 'candy', 'cannot', 'careful', 'carries', 'clamp', 'cloth', 'coaxing', 'coloration', 'come', 'crisp', 'despite', 'device', 'did', 'dif', 'does', 'double', 'eliminate', 'enough', 'even', 'exactly', 'expected', 'expensive', 'filter', 'filters', 'for', 'frequencies', 'gets', 'goose', 'gooseneck', 'grape', 'great', 'had', 'here', 'high', 'hint', 'hold', 'honestly', 'if', 'in', 'is', 'it', 'job', 'just', 'keep', 'lets', 'like', 'little', 'looks', 'lowest', 'marginally', 'may', 'metal', 'mic', 'might', 'mike', 'mine', 'more', 'mount', 'much', 'mxl', 'my', 'neck', 'needed', 'needs', 'next', 'nice', 'no', 'nose', 'not', 'noticeable', 'now', 'of', 'old', 'on', 'one', 'ones', 'only', 'or', 'otherwise', 'out', 'pass', 'performs', 'pleasing', 'pop', 'popping', 'pops', 'position', 'position

In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
document_tfidf_matrix = tfidf.fit_transform(corpus)
print(sorted(tfidf.vocabulary_))
print(document_tfidf_matrix.todense())

['able', 'about', 'added', 'affordable', 'after', 'allowing', 'amazon', 'an', 'and', 'are', 'aroma', 'arrived', 'as', 'attached', 'attaches', 'avoid', 'better', 'block', 'blocks', 'bonus', 'breath', 'but', 'buy', 'candy', 'cannot', 'careful', 'carries', 'clamp', 'cloth', 'coaxing', 'coloration', 'come', 'crisp', 'despite', 'device', 'did', 'dif', 'does', 'double', 'eliminate', 'enough', 'even', 'exactly', 'expected', 'expensive', 'filter', 'filters', 'for', 'frequencies', 'gets', 'goose', 'gooseneck', 'grape', 'great', 'had', 'here', 'high', 'hint', 'hold', 'honestly', 'if', 'in', 'is', 'it', 'job', 'just', 'keep', 'lets', 'like', 'little', 'looks', 'lowest', 'marginally', 'may', 'metal', 'mic', 'might', 'mike', 'mine', 'more', 'mount', 'much', 'mxl', 'my', 'neck', 'needed', 'needs', 'next', 'nice', 'no', 'nose', 'not', 'noticeable', 'now', 'of', 'old', 'on', 'one', 'ones', 'only', 'or', 'otherwise', 'out', 'pass', 'performs', 'pleasing', 'pop', 'popping', 'pops', 'position', 'position

# Part 3 : Comparing two documents / Similarity Measures

## 3.1 Euclidean distance

We could try the Euclidean distance $||\vec{x}-\vec{y}||$  
What problems would we encounter with this? 

The euclidean distance goes up with the length of a document. Intuitively, duplicating each word in our bag of words generates a vector that points in exactly the same direction, however, the euclidean distance goes up. One solution is to normalize vectors before calculating the euclidean distance. Now increasing the length of a document does not change the Euclidean distance unless the direction of the term frequency vector changes. 

## 3.2 Cosine Similarity
Recall that for two vector $\vec{x}$ and $\vec{y}$ that $\vec{x} \cdot \vec{y} = ||\vec{x}|| ||\vec{y}|| \cos{\theta}$. And so,

$\frac{\vec{x} \cdot \vec{y} }{||\vec{x}|| ||\vec{y}||} = \cos{\theta}$

$\theta$ can only range from 0 to 90 degrees, because tf-idf vectors are non-negative. Therefore cos $\theta$ ranges from 0 to 1. Documents that are exactly identical will have $\cos{\theta} = 1$.
